In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt
import numpy as np
import pandas as pd
import operator
import json

In [ ]:
url = 'https://raw.githubusercontent.com/Ashwin-19/HW3/master/Tweets.csv?token=AJD3RIUGFMHVMMULM6O7GJ26POKIC'
dataset = pd.read_csv(url, error_bad_lines=False)

In [ ]:
# removing null columns

dataset = dataset.drop(labels='Unnamed: 0',axis=1)

for column in list(dataset.columns):
  if dataset[column].isnull().values.all(axis=0):
    dataset = dataset.drop(labels=column,axis=1)

In [ ]:
# initialize twint
import twint
client = twint.Config()
client.Store_object = True

# retrieve user specific features: followers, following, bio, verified, account age
UserData = []
count = 0
for index, row in dataset.iterrows():
  client.Username = row['username']
  twint.run.Lookup(client)
  try:
    UserData.append(twint.output.users_list[0])
  except Exception:
    UserData.append(-1)
  twint.output.clean_lists()

In [ ]:
User_df = {"followers":[],"following":[],"bio":[],"verified":[],"age":[],"user_location":[]}

for user in UserData:
  if user != -1:
    User_df['followers'].append(user.followers)
    User_df['following'].append(user.following)
    User_df['bio'].append(user.bio)
    User_df['verified'].append(user.is_verified)
    User_df['age'].append((datetime.now() - datetime.strptime(user.join_date,"%d %b %Y")).days)
    User_df['user_location'].append(user.location)
  else:
    User_df['followers'].append(np.nan)
    User_df['following'].append(np.nan)
    User_df['bio'].append(np.nan)
    User_df['verified'].append(np.nan)
    User_df['age'].append(np.nan)
    User_df['user_location'].append(np.nan)

In [ ]:
dataset['followers'] = User_df['followers']
dataset['following'] = User_df['following']
dataset['bio'] = User_df['bio']
dataset['verified'] = User_df['verified']
dataset['age'] = User_df['age']
dataset['user_location'] = User_df['user_location']

In [ ]:
'''
Sentiment Analysis
-1 (-ve) -> 1 (+ve)
'''
from textblob import TextBlob
import re

sentiment_df = {"sentiment":[]}

for index, row in dataset.iterrows():
  tweet = row['tweet']
  text =  ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(tweet)).split())
  polarity = TextBlob(text).sentiment.polarity
  sentiment_df["sentiment"].append(polarity)

sentiment_df = pd.DataFrame(sentiment_df)
dataset = pd.concat([dataset, sentiment_df], axis=1, sort=False)
dataset.head()

,id,conversation_id,created_at,date,timezone,tweet,hashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,reply_to,plabel,ilabel,followers,following,bio,verified,age,user_location,sentiment
0,1.235145e+18,1.235145e+18,1583316412000,2020-03-04 10:06:52,UTC,#DelhiViolance: Cops arrest man seen firing in...,"['#delhiviolance', '#jaffrabad', '#savedelhihi...",1.057575e+18,1.057575e+18,news_khulasa,Khulasa News,6,10,https://twitter.com/news_khulasa/status/123514...,False,1,0,0,NaN,"[{'user_id': '1057575335280046080', 'username'...",report,fact,1517.0,1491.0,Official Twitter account of khulasa-news. We p...,0.0,506.0,,-0.05
1,1.234889e+18,1.234747e+18,1583255487000,2020-03-03 17:11:27,UTC,Police have asked people to don't come but the...,[],2.159009e+08,2.159009e+08,annuu143,annu kumar singh,1,6,https://twitter.com/annuu143/status/1234889136...,False,0,4,0,NaN,"[{'user_id': '215900941', 'username': 'annuu14...",other,chaos,10.0,55.0,,0.0,3414.0,,0.25
2,1.234812e+18,1.234812e+18,1583237170000,2020-03-03 12:06:10,UTC,"Mohammed Shahrukh, the man who aimed a gun at ...","['#delhiviolence', '#uttarpradesh']",2.264849e+09,2.264849e+09,LogicalIndians,The Logical Indian,6,6,https://twitter.com/LogicalIndians/status/1234...,False,15,2,2,NaN,"[{'user_id': '2264849412', 'username': 'Logica...",report,fact,81439.0,1967.0,"Handpicked, newsworthy stories which deserve t...",1.0,2275.0,India,0.00
3,1.234798e+18,1.234798e+18,1583233754000,2020-03-03 11:09:14,UTC,Delhi shooter Shahrukh (not @iamsrk )who point...,[],7.502518e+07,7.502518e+07,Mohit4Nation,Mohit Saxena,1,17,https://twitter.com/Mohit4Nation/status/123479...,False,0,0,0,NaN,"[{'user_id': '75025176', 'username': 'Mohit4Na...",praise,fact,104.0,98.0,NATIONALIST| STRATEGIST| FOLLOWS THE PRINCIPLE...,0.0,3837.0,India,0.00
4,1.234739e+18,1.234739e+18,1583219760000,2020-03-03 7:16:00,UTC,Ab iski tabiyat se kutai karein @DelhiPolice \...,[],7.815867e+17,7.815867e+17,kalpana0402,🕉 𝕮𝖔𝖚𝖈𝖍 𝕻𝖔𝖙𝖆𝖙𝖔 🇮🇳,7,18,https://twitter.com/kalpana0402/status/1234739...,False,1,0,0,NaN,"[{'user_id': '781586734823845892', 'username':...",action,fact,480.0,352.0,🇮🇳Nationalist|Hindu|Bibliophile/Melophile| Pur...,0.0,1268.0,,0.00


In [ ]:
dataset.to_csv('preprocessedDB.csv')